In [6]:
import pandas as pd
from collections import Counter
import stanza

# Načtení českého modelu Stanza
stanza.download('cs')  # Stáhne český model, pokud ještě není nainstalován
nlp = stanza.Pipeline('cs')

# Definice množiny alkoholických nápojů v základním tvaru (lemma)
chlast = {
    "pivo", "víno", "vodka", "rum", "whisky", "tequila", "gin", "šampaňské",
    "cider", "likér", "brandy", "koňak", "prosecco", "saké", "mezcal",
    "vermut", "absint", "bourbon", "šnaps"
}

# Funkce pro analýzu textu a detekci alkoholických nápojů
def find_alcoholic_words(text, chlast_lemmas):
    found_words = []
    doc = nlp(text)
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.lemma in chlast_lemmas:  # Kontrola lemmat
                found_words.append(word.text.lower())
    return found_words

# Funkce pro zpracování jednoho chunku
def process_chunk(chunk, all_alcohol_words, chlast_lemmas):
    for plot in chunk['plot_text']:
        if isinstance(plot, str):  # Ověření, zda je text platný string
            found = find_alcoholic_words(plot, chlast_lemmas)
            all_alcohol_words.extend(found)

# Inicializace seznamu pro sběr výsledků
all_alcohol_words = []

# Načítání souboru po částech (chunksize) a pouze sloupce 'plot_text'
chunk_size = 10000  # Velikost chunku
for chunk in pd.read_csv('CSFD_cleaned.csv', usecols=['plot_text'], chunksize=chunk_size):
    process_chunk(chunk, all_alcohol_words, chlast)

# Spočítání výskytů jednotlivých alkoholů
alcohol_count = Counter(all_alcohol_words)

# Celkový počet výskytů alkoholických nápojů
total_alcohol_words = sum(alcohol_count.values())

# Výpočet procent pro každý alkohol, zaokrouhleno na jedno desetinné místo
if total_alcohol_words > 0:
    alcohol_percentages = {alcohol: round((count / total_alcohol_words) * 100, 1) for alcohol, count in alcohol_count.items()}
else:
    alcohol_percentages = {}

# Vytvoření DataFrame pro lepší čitelnost
alcohol_percentages_df = pd.DataFrame(list(alcohol_percentages.items()), columns=['Alcohol', 'Percentage'])
alcohol_percentages_df.sort_values(by='Percentage', ascending=False, inplace=True)

# Uložení výsledků do CSV
alcohol_percentages_df.to_csv('filmy_CSFD_chlast.csv', index=False)

# Zobrazení výsledků
print(alcohol_percentages_df)


2024-11-08 23:42:34 INFO: Downloaded file to C:\Users\danto\stanza_resources\resources.json
2024-11-08 23:42:34 INFO: Downloading default packages for language: cs (Czech) ...
2024-11-08 23:42:36 INFO: File exists: C:\Users\danto\stanza_resources\cs\default.zip
2024-11-08 23:42:39 INFO: Finished downloading models and saved to C:\Users\danto\stanza_resources
2024-11-08 23:42:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-11-08 23:42:39 INFO: Downloaded file to C:\Users\danto\stanza_resources\resources.json
2024-11-08 23:42:40 INFO: Loading these models for language: cs (Czech):
| Processor | Package      |
----------------------------
| tokenize  | pdt          |
| mwt       | pdt          |
| pos       | pdt_nocharlm |
| lemma     | pdt_nocharlm |
| depparse  | pdt_nocharlm |

2024-11-08 23:42:40 INFO: Using device: cpu
2024-11-08 23:42:40 INFO: Loading: tokenize
c:\Users\danto\AppData\Local\Programs\Python\Python312\Lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import stanza

#stáhnu si češtinu stanzy
stanza.download('cs')

# inicializuje zpracovatelský kanál pro češtinu, který analyzuje text na věty, slova, slovní druhy, lemmata a syntaktické vztahy
nlp = stanza.Pipeline('cs')

# slova v češtině musí být mužský rod
vanoce = [
    'vánoční', 'Vánoce'
]

# Funkce, která vytáhne barvy z textu
def extract_christmas(text):
    if not isinstance(text, str): #Funkce zajišťuje, že vstup text je skutečně řetězec (str).
        return {}  # zda je vstup text typu řetězec, a pokud není, vrátí prázdný slovník (slovník protože barva: počet)
    
    doc = nlp(text) #Zpracuje vstupní text pomocí Stanza NLP pipeline a vytvoří objekt doc, který obsahuje informace o větách, slovech, jejich základních tvarech (lemmata) a dalších vlastnostech
    lemmas = [word.lemma for sentence in doc.sentences for word in sentence.words] #Extrahuje lemmata (základní tvary slov) ze všech vět a slov v textu a uloží je do seznamu lemmas.
    color_count = {} #Prázdný slovník, který bude ukládat jednotlivé barvy jako klíče a jejich počet výskytů jako hodnoty
    for lemma in lemmas: #uplně klasické nahazování do slovníku
        if lemma in vanoce:
            if lemma in color_count:
                color_count[lemma] += 1
            else:
                color_count[lemma] = 1
    return color_count #vrací počet barev

# Input and output file 
input_file = "CSFD_cleaned_final.csv"
output_file = "CSFD_colors_extracted.csv"

#Jdeme zpracovávat po chunkách
chunksize = 10000  # nastavuje velikost čuníka, který bude zpracovaný za účelem šetření paměti
chunk_list = []   #prázdný seznam, jakási ohrádka pro čuníky nebo chlívek

# Načítání souboru po čuníkách
for chunk in pd.read_csv(input_file, chunksize=chunksize): #načtení čuníka z obrovského chlívku CSV
    chunk['color_counts'] = chunk['plot_text'].apply(extract_christmas) #Čuníka krmíme, aby se dozvěděl, kolik barev v sobě má (počítáme barvy ve sloupci plot_text).
    chunk_list.append(chunk) # po zpracování jde čuník do ohrádky za kamarády a čeká tam na ostatní

# Spojení všech zpracovaných částí do jednoho DataFrame, Všechny čuníky spojíme do jednoho velkého stáda DataFrame
result_df = pd.concat(chunk_list)

#uložit dataframe do CSV
result_df.to_csv(output_file, index=False)


In [1]:
import pandas as pd
import stanza
from collections import Counter  # Import Counter

# Stažení modelu češtiny pro Stanzu
stanza.download('cs')

# Inicializace NLP pipeline pro češtinu
nlp = stanza.Pipeline('cs', processors='tokenize,lemma', use_gpu=True)

# Seznam vámi specifikovaných lemmat
vanoce = ['vánoční', 'Vánoce']

# Načtení dat po částech
input_file = "CSFD_cleaned_final.csv"
chunksize = 10000  # Velikost jednoho chunku

# Funkce pro extrakci lemmat
def extract_lemmas(text):
    if not isinstance(text, str):  # Pokud není text řetězec, vrátíme prázdný seznam
        return []
    doc = nlp(text)  # Zpracování textu pomocí NLP pipeline
    lemmas = [word.lemma for sentence in doc.sentences for word in sentence.words]  # Extrakce lemmat
    return lemmas

# Inicializace Counter pro kumulaci výsledků
global_lemma_counts = Counter()

# Zpracování po částech
for chunk in pd.read_csv(input_file, chunksize=chunksize):
    # Extrakce lemmat pro každý řádek v chunku
    chunk['lemmas'] = chunk['plot_text'].apply(extract_lemmas)
    
    # Filtrace relevantních lemmat
    chunk_lemmas = [lemma for lemmas_list in chunk['lemmas'] for lemma in lemmas_list if lemma in vanoce]
    
    # Aktualizace globálního počítadla
    global_lemma_counts.update(chunk_lemmas)

# Vytvoření DataFrame s výsledky
lemma_df = pd.DataFrame(global_lemma_counts.items(), columns=['lemma', 'count']).sort_values(by='count', ascending=False)

# Uložení výsledků do CSV
lemma_df.to_csv("lemma_frequencies_vanoce.csv", index=False)

print("Lemmatizace a počítání dokončeno. Výstup uložen do 'lemma_frequencies_vanoce.csv'.")


2024-11-13 15:44:50 INFO: Downloaded file to C:\Users\danto\stanza_resources\resources.json
2024-11-13 15:44:50 INFO: Downloading default packages for language: cs (Czech) ...
2024-11-13 15:44:54 INFO: File exists: C:\Users\danto\stanza_resources\cs\default.zip
2024-11-13 15:44:56 INFO: Finished downloading models and saved to C:\Users\danto\stanza_resources
2024-11-13 15:44:56 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-11-13 15:44:57 INFO: Downloaded file to C:\Users\danto\stanza_resources\resources.json
2024-11-13 15:44:57 WARNING: Language cs package default expects mwt, which has been added
2024-11-13 15:44:57 INFO: Loading these models for language: cs (Czech):
| Processor | Package      |
----------------------------
| tokenize  | pdt          |
| mwt       | pdt          |
| lemma     | pdt_nocharlm |

2024-11-13 15:44:57 WARNING: GPU requested, but is not available!
2024-11-13 15:44:57 INFO: Using device: cpu
2024-11-13 15:44:57 INFO: Loading: tokenize
c:\Users\danto\AppData\Local\Programs\Python\Python312\Lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models fo

Lemmatizace a počítání dokončeno. Výstup uložen do 'lemma_frequencies_vanoce.csv'.
